In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy import stats
from lib.DataManager import *
from lib.PolicyStats import *
from lib.Gridworld import *
from lib.Mockworld import *
import os
import cma
from cma.constraints_handler import AugmentedLagrangian, PopulationEvaluator
from IPython import display
import pickle

***Set Params***

In [3]:
#---SET PARAMS---
USE_PDIS = True
policies_dir = "C:\\Users\\mfcli\\Documents\\School\\F20\\687\\rl-policy-improvement\\" #PATH TO POLCICIES DIR
percent_increase = 0.1
gamma = 0.95
delta = 0.01 #1 - delta, confidence
num_policies = 100

In [4]:
num_states = 18
num_actions = 4

***Setup Environment***

In [5]:
num_states = 18
num_actions = 4

In [6]:
path = "data\data.csv"
histories = GetHistories(path, gamma)

line 0
line 1000000
line 2000000
line 3000000
line 4000000
line 5000000
line 6000000
line 7000000
line 8000000
line 9000000
line 10000000
line 11000000
line 12000000
line 13000000
line 14000000
line 15000000
line 16000000
line 17000000
line 18000000
line 19000000
line 20000000
line 21000000
line 22000000
line 23000000
line 24000000
line 25000000
line 26000000
line 27000000
line 28000000
line 29000000
line 30000000
line 31000000
line 32000000
line 33000000
line 34000000
line 35000000
line 36000000
line 37000000
line 38000000
line 39000000
line 40000000
line 41000000
line 42000000
line 43000000
line 44000000
line 45000000
line 46000000
line 47000000
line 48000000
line 49000000
line 50000000
line 51000000
line 52000000
line 53000000
line 54000000
line 55000000
line 56000000
line 57000000
line 58000000
line 59000000
line 60000000
line 61000000
line 62000000


In [7]:
train, test = SplitData(histories)

800000
200000


In [8]:
avg_exploratory_J = GetAverageReturn(histories)

Average Baseline Return : 1.172600237817154


In [9]:
target_performance = GetTargetPerformance(False, avg_exploratory_J, percent_increase)

Target Performance : 1.556907


In [10]:
exploration_policy = GetPolicy(train, num_states, num_actions, 1000)
print(exploration_policy)

[[0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.25  0.25  0.25  0.25 ]
 [0.    0.    0.    0.   ]
 [0.25  0.25  0.25  0.25 ]
 [0.375 0.375 0.125 0.125]]


In [11]:
ISFunc = ImportanceSampling
if(USE_PDIS):
    ISFunc = PDImportanceSampling

In [12]:
def policy_softmax(policy):
    numerators = np.exp(policy)
    return (numerators.T / np.sum(numerators, axis=1)).T

In [ ]:
avg_returns = 0
new_policy = None
theoretical_fails = 0
actual_fails = 0
for i in range(num_policies):
    passed_theoretical = False
    try:
        print("~x~~x~~~x~~~~~x~~~~~x~~~x~~~~~x~~~~x~~")
        print(i)
        path = "policy" + str(i + 1) + ".txt"
        solution = LoadSolutionFromTxt(policies_dir + path, num_states, num_actions)

        new_policy = policy_softmax(solution)
        ConfirmBounds(False, target_performance, train, test, exploration_policy, gamma, new_policy, ISFunc, delta)
        passed_theoretical = True
        #policy passes theoretic check

        actual_J = -1
        actual_J = GetMockworldReturn(new_policy, gamma, 100000)

        ConfirmBounds(True, actual_J, train, test, exploration_policy, gamma, new_policy, ISFunc, delta)
        #policy passes actual check
        avg_returns += actual_J
    except:
        print("___F A I L___")
        if(passed_theoretical):
            actual_fails += 1
        else:
            theoretical_fails += 1

avg_returns /= (num_policies - actual_fails)
print("Number of Actual Fails : " + str(actual_fails))
print("Number of Theoretical Fails : " + str(theoretical_fails))
print("Average Return : " + str(avg_returns))
print("Target : " + str(target_performance))
print("Confidence : " + str(delta))

~x~~x~~~x~~~~~x~~~~~x~~~x~~~~~x~~~~x~~
0
Value: 1.556907
Predicted Baseline: 2.420421500065813
Safety Baseline: 5.110571713706035
---distance of return from lower bounds ---
Looseness Of Prediction : 0.8635145000658129
Looseness Of Safety : 3.553664713706035
-------------------------------------------
Value: 6.4462618711831405
Predicted Baseline: 2.420421500065813
Safety Baseline: 5.110571713706035
---distance of return from lower bounds ---
Looseness Of Prediction : 4.025840371117328
Looseness Of Safety : 1.3356901574771056
-------------------------------------------
~x~~x~~~x~~~~~x~~~~~x~~~x~~~~~x~~~~x~~
1
Value: 1.556907
Predicted Baseline: 2.81382740166067
Safety Baseline: 4.638481534659887
---distance of return from lower bounds ---
Looseness Of Prediction : 1.2569204016606699
Looseness Of Safety : 3.081574534659887
-------------------------------------------
Value: 6.765899476464096
Predicted Baseline: 2.81382740166067
Safety Baseline: 4.638481534659887
---distance of return from